In [103]:
import sqlite3
from sqlite3 import Error
import pandas as pd
from Database import PySQLite
from CorporateCredit import CorporateCredit
import math
import blpapi
import pdblp
from datetime import date

class CompareCredits():
    # all functions are comparing credits in single table
    # an improvement would be applying accross tables using join()'s
    #e.g. All 30y bonds = all tables, BB credits, etc 
    
    def __init__(self,dbname):
        self.DB = PySQLite(dbname)
        self.TargetDict={'FiveISIN':'FiveTarget','SevenISIN':'SevenTarget','TenISIN':'TenTarget',
                    'ThirtyISIN':'ThirtyTarget','PerpISIN':'PerpTarget'}
        
#         self.dbname=dbname
#         conn = sqlite3.connect(self.dbname)
#         c = conn.cursor()
#         conn.close()

    def Market_Data(self,ISIN,field):
        con = pdblp.BCon(debug=False,port=8194,timeout=5000)
        con.start()

        # df=con.bdh(self.ticker+" equity",'px_last','20201211','20201214')
        # print(df.index)
#         cusip = self.benchmarkdict[bond]
        df = con.ref(ISIN,field)
        return(df.iloc[0]['value'])
    

        
        
    def CreditTargets(self,table,security,field):
        
        #returns levels vs targets for a credits in a single table
        df = self.DB.DBdescription(table)
        df = df.iloc[:-1] #cutoff last row bc of placeholder         

        corplist=df[security].apply(lambda x: x+" corp")
    
        levels=corplist.apply(self.Market_Data,args=[field])
        difference=levels-df[self.TargetDict[security]]
        
        comps=pd.concat([df['Name'],df[self.TargetDict[security]],levels,difference],axis=1,keys=['Name','Target','Actual','Up/(Down)'])
        comps_sorted=comps.sort_values(by=['Up/(Down)'],ascending=False)
        index=comps_sorted.index
        index.name=security
        return(comps_sorted)
    
#     def BestCredit(self,table,security,field):
#         #returns best credit (highest upside) in a given table
#         may not need
#         True 
#         for 30y - 
#         max=ThirtyComps['Up/(Down)'].idxmax()
#         print(ThirtyComps.loc[max]['Name'])
        

            
            
            
def Main():    
    Comp=CompareCredits('test1228v1.db')
    ThirtyComps=Comp.CreditTargets('test_table1','ThirtyISIN','BLOOMBERG_MID_G_SPREAD')
    FiveComps=Comp.CreditTargets('test_table1','FiveISIN','BLOOMBERG_MID_G_SPREAD')
#     test=ThirtyComps.sort_values(by=['Up/(Down)'])
#     print(test)
#     print(type(ThirtyComps.iloc[2]['Actual']))
#     print(ThirtyComps['Actual'] - ThirtyComps['Target'])
    print(ThirtyComps)
    print(FiveComps)
    
    
    


Main()
           
            

             Name  Target    Actual  Up/(Down)
ThirtyISIN                                    
3             WES   320.0  425.5483   105.5483
4            ENLK   480.0  548.2336    68.2336
5           ONEOK   200.0  246.8513    46.8513
2             DCP   330.0  373.5537    43.5537
0             ETP   250.0  285.2750    35.2750
1             PAA   270.0  284.9764    14.9764
           Name  Target    Actual  Up/(Down)
FiveISIN                                    
3           WES   150.0  336.2022   186.2022
2           DCP   200.0  272.1466    72.1466
5         ONEOK    80.0  124.5109    44.5109
4          ENLK   480.0  496.1750    16.1750
1           PAA   145.0  151.0026     6.0026
0           ETP   115.0  117.5370     2.5370
WES
